In [1]:
import json
import string
import random 
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
from nltk.tag import pos_tag
import tensorflow as tensorF
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

2022-12-11 21:29:30.493312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mumtazcemeris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mumtazcemeris/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
ner = en_core_web_sm.load()

In [3]:
from pprint import pprint
# NER (PERSON) test

doc = ner("John's grades are fantastic and I would like to congralation him")
pprint([(X.text, X.label_) for X in doc.ents])
for x in doc.ents:
    if x.label_ == "PERSON":
        print(f"person is found: {x.text}")
doc = ner("His grades are fantastic and I would like to congralation him")
pprint([(X.text, X.label_) for X in doc.ents])
for x in doc.ents:
    if x.label_ == "PERSON":
        print(f"person is found: {x.text}")

[('John', 'PERSON')]
person is found: John
[]


In [4]:
#"patterns": ["Joe's.", "John's", "Melinda's", "Angelina's", "Josh's", "John's.","Brad's grades.", "Matthew's grades.", "Fatma's grades.", "Ayse's grades.", "Ali's grades.", "John's grades"],

ourData = {"intents": [
            
             {"tag": "enter_grade",
              "patterns": ["I want to enter the grades.", "Enter the grades.", "I would like to enter the grades.", "Enter", "entering"],
              "responses": ["Whose grades would you like to Flow?"]
             },
              {"tag": "query_grade",
              "patterns": [ "I want to query the grades.", "Query the grades.", "I would like to query the grades.", "query", "querying"],
              "responses": ["Whose grades would you like to Flow?"],
             },
              {"tag": "end",
              "patterns": [ "No.", "No, thanks.", "End it", "Finish"],
              "responses": ["I can help you either enter the grades of the students or query the grades of the students. Which one do you prefer?"]
             },
              {"tag": "continue",
              "patterns": [ "Yes.", "Yes, please"],
              "responses": ["Okay, whose grades would you like to Flow?"]
             },
             {"tag": "invalid_answer",
              "patterns": ["qweqweqw"],
              "responses": ["Sorry, can not understand you. Try again."]
             },
             {"tag": "person_flow_grade",
              "patterns": ["Someone", "Someone's.", "Someone's grades."],
              "responses": ["Which of the following grade do you want to Flow: Homework1, Homework2, Midterm?"]
             },
             {"tag": "enter_grade_type",
              "patterns": ["I will enter Homework1.", "I will enter Homework2.", "I will enter Midterm.", "Midterm"],
              "responses": ["What is the grade of Someone for Grade_type?"]
             },
             {"tag": "query_grade_type",
              "patterns": ["I will query Homework1.", "I will query Homework2.", "I will query Midterm."],
              "responses": ["Ok. Someone's Grade_type grade is Grade_num. Would you like to query another grade?"]
             },
             {"tag": "grade",
              "patterns": ["Grade_num"],
              "responses": ["I have entered the following grade: Someone’s Grade_type grade is Grade_num. Would you like to enter another grade?"]
             }
            
]}

In [5]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent["tag"])
    
    
    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
print(newWords)
ourClasses = sorted(set(ourClasses))# sorting classes

["'s", 'end', 'enter', 'entering', 'finish', 'grade', 'grade_num', 'homework1', 'homework2', 'i', 'it', 'like', 'midterm', 'no', 'please', 'query', 'querying', 'qweqweqw', 'someone', 'thanks', 'the', 'to', 'want', 'will', 'would', 'yes']


In [6]:
print(ourClasses)

['continue', 'end', 'enter_grade', 'enter_grade_type', 'grade', 'invalid_answer', 'person_enter_grade', 'query_grade', 'query_grade_type']


In [7]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# BOW model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)
    
    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])
print(trainingData)
random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = num.array(list(trainingData[:, 0]))# first trainig phase
y = num.array(list(trainingData[:, 1]))# second training phase

[[[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0]], 

In [8]:
# defining some parameters
iShape = (len(x[0]),)
oShape = len(y[0])

# the deep learning model
ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(Dropout(0.5))
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tensorF.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
#ourNewModel.fit(x, y, epochs=200, verbose=1)
ourNewModel.fit(x, y, epochs=200)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               3456      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 12,297
Trainable params: 12,297
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200


2022-12-11 21:34:18.940832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 362ms/step - loss: 2.1788 - accuracy: 0.2500
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 2.0119 - accuracy: 0.3214
Epoch 3/200
1/1 [==============================] - 0s 5ms/step - loss: 1.8876 - accuracy: 0.3571
Epoch 4/200
1/1 [==============================] - 0s 5ms/step - loss: 1.7319 - accuracy: 0.3929
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 1.5957 - accuracy: 0.5357
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - loss: 1.4491 - accuracy: 0.6786
Epoch 7/200
1/1 [==============================] - 0s 4ms/step - loss: 1.3370 - accuracy: 0.6071
Epoch 8/200
1/1 [==============================] - 0s 4ms/step - loss: 1.2097 - accuracy: 0.6429
Epoch 9/200
1/1 [==============================] - 0s 4ms/step - loss: 0.9674 - accuracy: 0.8571
Epoch 10/200
1/1 [==============================] - 0s 4ms/step - loss: 0.8873 - accuracy: 0.7143
Epoch 11/200
1/1 [=====================

1/1 [==============================] - 0s 4ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 4ms/step - loss: 2.5817e-04 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 3ms/step - loss: 2.4901e-04 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 3ms/step - loss: 5.2114e-04 - accuracy: 1.0000
Epoch 92/200
1/1 [==============================] - 0s 3ms/step - loss: 6.0236e-04 - accuracy: 1.0000
Epoch 93/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 94/200
1/1 

1/1 [==============================] - 0s 3ms/step - loss: 8.2020e-04 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 3ms/step - loss: 5.0379e-04 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1800 - accuracy: 0.9643
Epoch 168/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0932 - accuracy: 0.9643
Epoch 169/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 3ms/step - loss: 1.4129e-04 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 3ms/step - loss: 1.8763e-04 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0416 - accuracy: 0.9643
Epoch 17

In [9]:
grade_types = ["Homework1", "Homework2", "Midterm"]

def isPersonFound(text):
    doc = ner(text)
    for x in doc.ents:
        if x.label_ == "PERSON":
            print(f"person is found: {x.text}")
            return True, x.text
    return False, ""

def ourText(text): 
    newtkns = nltk.word_tokenize(text)
    newtkns = [lm.lemmatize(word) for word in newtkns]
    return newtkns

def wordBag(text, vocab): 
    newtkns = ourText(text)
    bagOwords = [0] * len(vocab)
    for w in newtkns: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bagOwords[idx] = 1
    return num.array(bagOwords)

def pred_class(text, vocab, labels): 
    bagOwords = wordBag(text, vocab)
    ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
    newThresh = 0.2
    yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

    yp.sort(key=lambda x: x[1], reverse=True)
    newList = []
    for r in yp:
        newList.append(labels[r[0]])
    return newList

def getRes(firstlist, fJson, personName, flow_type, current_grade, grade_type): 
    tag = firstlist[0]
    listOfIntents = fJson["intents"]
    for i in listOfIntents: 
        if i["tag"] == tag:
            ourResult = random.choice(i["responses"])
            break
    ourResult = ourResult.replace("Someone", personName)
    ourResult = ourResult.replace("Flow", flow_type)
    ourResult = ourResult.replace("Grade_num", current_grade)
    ourResult = ourResult.replace("Grade_type", grade_type)
    return ourResult

In [14]:
print("HW2: Hello my name is HW2. I can help you either enter the grades of the students or query the grades of the students. Which one do you prefer?")
# dialogue managing via entities below
personName = ""
flow_type = "" # either enter or query grade
current_grade = ""
grade_type = ""
students_db = []
while True:
    newMessage = input("User: ")
    isFound, person_name = isPersonFound(newMessage)
    
    if isFound and personName == "":
        personName = person_name
        newMessage = newMessage.replace(person_name,"Someone")
    elif isFound and personName != person_name:
        print("HW2: Aborting operation, you entered different person name!")
        print("HW2: I can help you either enter the grades of the students or query the grades of the students. Which one do you prefer?")
        personName = ""
        flow_type = ""
        continue
    elif not isFound and personName == "" and flow_type != "": 
        print("HW2: Please enter the name of the person!")
        continue
    
    intents = pred_class(newMessage, newWords, ourClasses)
    
    if newMessage.isdigit():
        current_grade = newMessage
        newMessage = newMessage.replace(current_grade,"Grade_num")
        intents[0] = "grade"
    print(f"intent is {intents[0]}")
    if intents[0] == 'end':
        # reset
        personName = ""
        flow_type = ""
        print(f"This is the result: {students_db}")
        print("HW2: I can help you either enter the grades of the students or query the grades of the students. Which one do you prefer?")

    elif intents[0] == 'continue':
        personName = ""
    elif intents[0] == 'enter_grade':
        flow_type = "enter"
    elif intents[0] == 'query_grade':
        flow_type = "query"
    elif intents[0] == 'grade':
        # check if student exists
        exists = False
        for student in students_db:
            if list(student.keys())[0] == personName:
                if grade_type == "Homework1":
                    student[personName][0] = current_grade
                    exists = True
                elif grade_type == "Homework2":
                    student[personName][1] = current_grade
                    exists = True
                elif grade_type == "Midterm":
                    student[personName][2] = current_grade
                    exists = True
                else:
                    print("Please enter valid grade type!")
                    continue
        if not exists:
            # create a new student
            stud_dict = {}
            stud_dict[personName] = ["", "", ""]
            if grade_type == "Homework1":
                stud_dict[personName][0] = current_grade
            elif grade_type == "Homework2":
                stud_dict[personName][1] = current_grade
            elif grade_type == "Midterm":
                stud_dict[personName][2] = current_grade
            students_db.append(stud_dict)
    elif intents[0] == 'enter_grade_type':
        for el in grade_types:
            if el in newMessage:
                grade_type = el
        if grade_type == "":
            print("Please enter valid grade type!")
            continue
    elif intents[0] == 'query_grade_type':
        for el in grade_types:
            if el in newMessage:
                grade_type = el
        if grade_type == "":
            print("Please enter valid grade type!")
            continue
        response_grade = -1
        grade_type_index = -1
        if grade_type == "Homework1":
            grade_type_index = 0
        elif grade_type == "Homework2":
            grade_type_index = 1
        elif grade_type == "Midterm":
            grade_type_index = 2
        for student in students_db:
            if list(student.keys())[0] == personName:
                response_grade = student[personName][grade_type_index] = current_grade
        if response_grade == -1:
            # student could not be found
            print("Please enter valid Student!")
            continue 
        else:
            current_grade = response_grade
            
    ourResult = getRes(intents, ourData, personName, flow_type, current_grade, grade_type)
    print(f"HW2: {ourResult}")
    

HW2: Hello my name is HW2. I can help you either enter the grades of the students or query the grades of the students. Which one do you prefer?
User: I want to enter the grades
1/1 [==============================] - 0s 19ms/step
intent is enter_grade
HW2: Whose grades would you like to enter?
User: John's
person is found: John
1/1 [==============================] - 0s 19ms/step
intent is person_enter_grade
HW2: Which of the following grade do you want to enter: Homework1, Homework2, Midterm?
User: I will enter Homework1
1/1 [==============================] - 0s 18ms/step
intent is enter_grade_type
HW2: What is the grade of John for Homework1?
User: 95
1/1 [==============================] - 0s 17ms/step
intent is grade
HW2: I have entered the following grade: John’s Homework1 grade is 95. Would you like to enter another grade?
User: Yes
1/1 [==============================] - 0s 17ms/step
intent is continue
HW2: Okay, whose grades would you like to enter?
User: John's
person is found: Jo

KeyboardInterrupt: Interrupted by user

In [ ]:
students_db

In [ ]:
db = [{"Cem": ["95","75", ""]}, {"Lara": ["100","100",""]}]
for el in db:
    print(list(el.keys())[0])
    print(el["Cem"][1])

In [ ]:
db = []
for el in db:
    print(list(el.keys())[0]=="Cem")